In [1]:
import dill
from copy import deepcopy
import time
import random
import numpy as np
from sklearn.metrics import roc_curve, auc

import nltk

nltk.download("punkt")
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn

from torchtext.legacy.data import Field
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import BucketIterator
from torchtext.legacy.data import Iterator

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amole\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

DATA_PATH = "data/processed/"

In [3]:
# 데이터 불러오기
TEXT = Field(
    sequential=True,
    use_vocab=True,
    tokenize=word_tokenize,
    lower=True,
    batch_first=True,
)
LABEL = Field(
    sequential=False,
    use_vocab=False,
    batch_first=True,
)


cola_train_data, cola_valid_data, cola_test_data = TabularDataset.splits(
    path=DATA_PATH,
    train="cola_train.tsv",
    validation="cola_valid.tsv",
    test="cola_test.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1
)

TEXT.build_vocab(cola_train_data, min_freq=2)


cola_train_iterator, cola_valid_iterator, cola_test_iterator = BucketIterator.splits(
    (cola_train_data, cola_valid_data, cola_test_data), 
    batch_size=32, 
    device=None,
    sort=False,
)


sat_train_data, sat_valid_data, sat_test_data = TabularDataset.splits(
    path=DATA_PATH,
    train="sat_train.tsv",
    validation="sat_valid.tsv",
    test="sat_test.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1
)

sat_train_iterator, sat_valid_iterator, sat_test_iterator = BucketIterator.splits(
    (sat_train_data, sat_valid_data, sat_test_data), 
    batch_size=8, 
    device=None,
    sort=False,
)

In [4]:
# LSTM Pooling Classifier
class LSTMPoolingClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super(LSTMPoolingClassifier, self).__init__()
        self.embed_layer = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=pad_idx)
        self.hidden_size = hidden_size
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        self.ih2h = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers,
                            bidirectional=True, batch_first=True, dropout=0.5)
        self.pool2o = nn.Linear(2 * hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout(p=0.5)


    def forward(self, x):
        x = self.embed_layer(x) # Token으로 들어온 데이터를 Embedding하여 값으로 변환
        o, _ = self.ih2h(x) #변환된 값들을 lstm에 넣는다
        pool = nn.functional.max_pool1d(o.transpose(1, 2), x.shape[1]) # LSTM의 결과를 Max Pooling
        pool = pool.transpose(1, 2).squeeze() # Max Pooling한 결과를 Fully Connected Layer에 넣기위해 Shape를 맞춰준다.
        output = self.sigmoid(self.pool2o(pool)) # Fully Connected Layer에 넣에서 결과를 반환
        return output.squeeze()

In [5]:
def train(model: nn.Module,
          iterator: Iterator,
          optimizer: torch.optim.Optimizer,
          criterion: nn.Module,
          device: str):
    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):
        optimizer.zero_grad()

        text = batch.text
        if text.shape[0] > 1:
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)

            output = model(text).flatten()
            loss = criterion(output, label)
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: Iterator,
             criterion: nn.Module,
             device: str):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for _, batch in enumerate(iterator):
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def test(
    model: nn.Module,
    iterator: Iterator,
    device: str):

    with torch.no_grad():
        y_real = []
        y_pred = []
        model.eval()
        for batch in iterator:
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)

            output = model(text).flatten().cpu()

            y_real += [label]
            y_pred += [output]

        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)

    fpr, tpr, _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)

    return auroc

def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [6]:
# 데이터 사전 학습(CoLA)
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20

lstm_pool_classifier = LSTMPoolingClassifier(
    num_embeddings=len(TEXT.vocab),
    embedding_dim=100,
    hidden_size=200,
    num_layers=4,
    pad_idx=PAD_IDX,
)

if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
_ = lstm_pool_classifier.to(device)

optimizer = torch.optim.Adam(lstm_pool_classifier.parameters())
bce_loss_fn = nn.BCELoss()

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(
        lstm_pool_classifier, 
        cola_train_iterator, # CoLA를 활용하여 사전학습
        optimizer, 
        bce_loss_fn, 
        device
    )
    valid_loss = evaluate(
        lstm_pool_classifier, 
        cola_valid_iterator, 
        bce_loss_fn, 
        device
    )

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.5f}')
    print(f'\t Val. Loss: {valid_loss:.5f}')

Epoch: 01 | Time: 0m 49s
	Train Loss: 0.60672
	 Val. Loss: 0.62315
Epoch: 02 | Time: 0m 49s
	Train Loss: 0.58989
	 Val. Loss: 0.62188
Epoch: 03 | Time: 0m 49s
	Train Loss: 0.56865
	 Val. Loss: 0.62514
Epoch: 04 | Time: 0m 50s
	Train Loss: 0.53407
	 Val. Loss: 0.62893
Epoch: 05 | Time: 0m 51s
	Train Loss: 0.49146
	 Val. Loss: 0.65067
Epoch: 06 | Time: 0m 55s
	Train Loss: 0.43319
	 Val. Loss: 0.73733
Epoch: 07 | Time: 0m 51s
	Train Loss: 0.36779
	 Val. Loss: 0.78601
Epoch: 08 | Time: 0m 49s
	Train Loss: 0.32446
	 Val. Loss: 0.83179
Epoch: 09 | Time: 0m 49s
	Train Loss: 0.26992
	 Val. Loss: 0.97053
Epoch: 10 | Time: 0m 49s
	Train Loss: 0.22730
	 Val. Loss: 1.18654
Epoch: 11 | Time: 0m 49s
	Train Loss: 0.18931
	 Val. Loss: 1.11717
Epoch: 12 | Time: 0m 49s
	Train Loss: 0.14701
	 Val. Loss: 1.33114
Epoch: 13 | Time: 0m 49s
	Train Loss: 0.12902
	 Val. Loss: 1.57492
Epoch: 14 | Time: 0m 50s
	Train Loss: 0.11716
	 Val. Loss: 1.41129
Epoch: 15 | Time: 0m 49s
	Train Loss: 0.10760
	 Val. Loss: 1.4

In [7]:
# 성능 비교를 위한 모델 저장
before_tuning_lstm_pool_classifier = deepcopy(lstm_pool_classifier)

In [8]:
# 수능 데이터를 이용한 추가학습(Fine-tuning)
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20


for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(lstm_pool_classifier, sat_train_iterator, optimizer, bce_loss_fn, device)
    valid_loss = evaluate(lstm_pool_classifier, sat_valid_iterator, bce_loss_fn, device)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.5f}')
    print(f'\t Val. Loss: {valid_loss:.5f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 0.69801
	 Val. Loss: 0.89476
Epoch: 02 | Time: 0m 2s
	Train Loss: 0.30098
	 Val. Loss: 0.81381
Epoch: 03 | Time: 0m 2s
	Train Loss: 0.22424
	 Val. Loss: 0.89976
Epoch: 04 | Time: 0m 2s
	Train Loss: 0.17000
	 Val. Loss: 1.04335
Epoch: 05 | Time: 0m 2s
	Train Loss: 0.07650
	 Val. Loss: 1.20248
Epoch: 06 | Time: 0m 2s
	Train Loss: 0.06741
	 Val. Loss: 1.30694
Epoch: 07 | Time: 0m 2s
	Train Loss: 0.10699
	 Val. Loss: 1.32122
Epoch: 08 | Time: 0m 2s
	Train Loss: 0.05806
	 Val. Loss: 1.20943
Epoch: 09 | Time: 0m 2s
	Train Loss: 0.07408
	 Val. Loss: 1.32051
Epoch: 10 | Time: 0m 1s
	Train Loss: 0.06617
	 Val. Loss: 1.52681
Epoch: 11 | Time: 0m 1s
	Train Loss: 0.06221
	 Val. Loss: 1.54331
Epoch: 12 | Time: 0m 2s
	Train Loss: 0.06098
	 Val. Loss: 1.57563
Epoch: 13 | Time: 0m 1s
	Train Loss: 0.05473
	 Val. Loss: 1.53700
Epoch: 14 | Time: 0m 2s
	Train Loss: 0.04116
	 Val. Loss: 1.54010
Epoch: 15 | Time: 0m 2s
	Train Loss: 0.05185
	 Val. Loss: 1.59747
Epoch: 16 

In [9]:
# 모델의 성능 비교
_ = before_tuning_lstm_pool_classifier.cpu()
_ = lstm_pool_classifier.cpu()

pool_sat_test_auroc = test(before_tuning_lstm_pool_classifier, sat_test_iterator, "cpu")
pool_tuned_test_auroc = test(lstm_pool_classifier, sat_test_iterator, "cpu")

print(f"Before fine-tuning SAT Dataset Test AUROC: {pool_sat_test_auroc:.5f}")
print(f"After fine-tuning SAT Dataset Test AUROC: {pool_tuned_test_auroc:.5f}")

Before fine-tuning SAT Dataset Test AUROC: 0.30769
After fine-tuning SAT Dataset Test AUROC: 0.30769


In [12]:
# 모델 저장
with open("advanced_before_tuning_model.dill", "wb") as f:
    model = {
        "TEXT": TEXT,
        "LABEL": LABEL,
        "classifier": before_tuning_lstm_pool_classifier
    }
    dill.dump(model, f)

with open("advanced_after_tuning_model.dill", "wb") as f:
    model = {
        "TEXT": TEXT,
        "LABEL": LABEL,
        "classifier": lstm_pool_classifier
    }
    dill.dump(model, f)